<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-05 13:00:11
-------------------
qualified stocks: 93
with latest results: 31
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.64 L
Current:  1.33 C
-------------------
Today PnL: 71.16 K (0.54%)
Current PnL: -15.98 L (-11.14%)
CY Booked + Current PnL: -10.99 L (-7.66%)
-------------------
Total profit:  4.33 L
Total loss:  -20.31 L
-------------------
Total Booked + Current PnL: 15.99 L (13.72%)
Total Booked PnL: 31.97 L (27.43%)
Curr Year Booked PnL: 4.99 L (3.76%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 78.93 L (59.51%)
-------------------
Est LTT:  2.74 C
Est LTT PnL: 1.41 C (106.37%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,785.05,983.0,-2.96,H-LC,88.17,237572.0,42880.0,6201.0,0.49,22.02,2.61,25.21,31.0,6.92,1.81,31.37,XY25,NTT,INSURANCE
62,RELIANCE,1291.83,1526.0,6.12,X-LC,29.03,157778.0,16969.0,8552.0,1.69,12.05,5.42,18.13,19.0,1.98,1.20,24.58,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.0,3.24,M-LC,62.37,103610.0,25248.0,8786.0,1.61,32.22,8.48,43.43,70.0,2.87,0.79,38.58,XY24,NTT,MISC
85,UNIONBANK,123.87,163.0,-0.31,M-LC,91.40,174132.0,33292.0,11197.0,1.00,23.64,6.43,31.59,79.0,2.97,1.33,51.26,XY24,NTT,BANKS
56,PFIZER,4245.50,6318.3,15.43,H-SC,98.92,133342.0,35696.0,11974.0,0.82,36.56,8.98,48.82,134.0,2.98,1.02,48.13,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SFL,1016.35,1287.0,286.53,H-SC,25.81,174537.0,-87681.0,157502.0,8.00,-33.44,90.24,26.63,136.0,-0.56,1.33,8.52,XY24,NTT,MISC
30,GREENPANEL,375.16,537.0,195.22,M-SC,82.80,131662.0,-51416.0,130398.0,3.17,-28.08,99.04,43.14,191.0,-0.39,1.01,23.85,XY24,NTT,MISC
72,SIS,477.00,528.0,2276.93,M-SC,87.10,64345.0,-16745.0,25416.0,3.05,-20.65,39.50,10.69,201.0,-0.66,0.49,30.40,X40N,NTT,MISC
84,UJJIVANSFB,52.77,60.0,38.86,M-SC,60.22,120204.0,-22275.0,41795.0,2.63,-15.63,34.77,13.70,221.0,-0.53,0.92,41.87,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-379.48,H-SC,100.00,118703.0,-14583.0,99176.0,2.17,-10.94,83.55,63.47,130.0,-0.15,0.91,98.98,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VIPIND,488.80,718.0,-819.40,H-SC,65.59,68533.0,-26783.0,71480.0,-1.93,-28.10,104.30,46.89,135.0,-0.37,0.52,35.17,X40N,NTT,MISC
59,RAJESHEXPO,517.65,583.0,1846.97,M-SC,52.69,53109.0,-84068.0,101385.0,-1.68,-61.28,190.90,12.62,209.0,-0.83,0.41,32.08,X40N,NTT,JEWELLERY
75,SURYODAY,189.78,240.0,60.09,H-SC,66.67,113839.0,-36087.0,75760.0,-1.56,-24.07,66.55,26.46,129.0,-0.48,0.87,45.84,XR,NTT,BANKS
70,SHALBY,261.39,327.0,820.26,M-SC,12.90,125156.0,-55987.0,101451.0,-1.40,-30.91,81.06,25.10,206.0,-0.55,0.96,0.17,XY24,NTT,HEALTHCARE
60,RBLBANK,230.61,388.0,37.52,M-SC,43.01,213390.0,-20910.0,180827.0,-1.15,-8.92,84.74,68.25,222.0,-0.12,1.63,40.53,XY24,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-13.14,X-LC,51.61,279007.0,17406.0,128204.0,0.48,6.65,45.95,55.67,1.0,0.14,2.13,13.16,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.05,X-LC,19.35,263741.0,-32193.0,126095.0,0.01,-10.88,47.81,31.73,2.0,-0.26,2.01,5.57,X40,BTT,IT
51,LTIM,5564.16,7197.33,-2.79,X-LC,79.57,187128.0,-13182.0,71969.0,0.87,-6.58,38.46,29.35,3.0,-0.18,1.43,29.28,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.26,X-LC,4.30,152701.0,-48685.0,136301.0,-0.15,-24.17,89.26,43.51,5.0,-0.36,1.17,5.32,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-15.50,X-LC,27.96,265675.0,-4672.0,39293.0,0.32,-1.73,14.79,12.81,6.0,-0.12,2.03,9.95,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PFIZER,4245.50,6318.3,15.43,H-SC,98.92,133342.0,35696.0,11974.0,0.82,36.56,8.98,48.82,134.0,2.98,1.02,48.13,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-0.31,M-LC,91.40,174132.0,33292.0,11197.0,1.00,23.64,6.43,31.59,79.0,2.97,1.33,51.26,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.24,M-LC,62.37,103610.0,25248.0,8786.0,1.61,32.22,8.48,43.43,70.0,2.87,0.79,38.58,XY24,NTT,MISC
50,LICI,785.05,983.0,-2.96,H-LC,88.17,237572.0,42880.0,6201.0,0.49,22.02,2.61,25.21,31.0,6.92,1.81,31.37,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,101.81,M-MC,86.02,229715.0,4753.0,162891.0,1.19,2.11,70.91,74.53,170.0,0.03,1.75,33.37,XY24,BTT,STEEL
52,LUXIND,1448.19,2442.00,40.00,M-SC,69.89,145350.0,7772.0,86643.0,0.29,5.65,59.61,68.62,183.0,0.09,1.11,26.01,X40N,NTT,TEXTILES
56,PFIZER,4245.50,6318.30,15.43,H-SC,98.92,133342.0,35696.0,11974.0,0.82,36.56,8.98,48.82,134.0,2.98,1.02,48.13,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.45,H-MC,47.31,102026.0,16925.0,26180.0,1.77,19.89,25.66,50.65,90.0,0.65,0.78,28.25,X40N,ATH,CABLES
18,CAMS,3643.00,5250.99,5.56,H-SC,70.97,118552.0,16548.0,28476.0,1.45,16.22,24.02,44.14,112.0,0.58,0.90,35.10,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,5.56,H-SC,70.97,118552.0,16548.0,28476.0,1.45,16.22,24.02,44.14,112.0,0.58,0.90,35.10,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-28.22,M-MC,58.06,141510.0,12358.0,74703.0,-0.43,9.57,52.79,67.42,156.0,0.17,1.08,37.63,XR,NTT,BANKS
39,INDIAMART,2327.09,4911.36,-50.91,H-SC,61.29,127645.0,4309.0,132661.0,1.20,3.49,103.93,111.05,108.0,0.03,0.97,25.07,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.16,M-LC,36.02,153971.0,3026.0,106425.0,0.60,2.00,69.12,72.51,68.0,0.03,1.18,12.19,XR,NTT,IT
1,AARTIIND,487.04,919.00,39.94,M-SC,59.14,80410.0,-926.0,73061.0,0.02,-1.14,90.86,88.69,205.0,-0.01,0.61,35.60,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.5,-25.02,H-SC,56.99,85425.0,-12555.0,34426.0,-0.82,-12.81,40.30,22.32,127.0,-0.36,0.65,23.90,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,3.24,M-LC,62.37,103610.0,25248.0,8786.0,1.61,32.22,8.48,43.43,70.0,2.87,0.79,38.58,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-379.48,H-SC,100.00,118703.0,-14583.0,99176.0,2.17,-10.94,83.55,63.47,130.0,-0.15,0.91,98.98,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,38.86,M-SC,60.22,120204.0,-22275.0,41795.0,2.63,-15.63,34.77,13.70,221.0,-0.53,0.92,41.87,X40N,NTT,BANKS
56,PFIZER,4245.50,6318.3,15.43,H-SC,98.92,133342.0,35696.0,11974.0,0.82,36.56,8.98,48.82,134.0,2.98,1.02,48.13,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-379.48,H-SC,100.00,118703.0,-14583.0,99176.0,2.17,-10.94,83.55,63.47,130.0,-0.15,0.91,98.98,XR,NTT,CHEMICALS
56,PFIZER,4245.50,6318.3,15.43,H-SC,98.92,133342.0,35696.0,11974.0,0.82,36.56,8.98,48.82,134.0,2.98,1.02,48.13,X40,ATH,PHARMA
8,ASIANTILES,75.41,137.0,6527.78,M-SC,97.85,72712.0,-21098.0,97718.0,0.76,-22.49,134.39,81.67,214.0,-0.22,0.56,40.81,XR,NTT,CERAMICS
81,TITAGARH,1097.23,1548.0,-6.68,H-SC,96.77,164036.0,-27979.0,106869.0,0.78,-14.57,65.15,41.08,128.0,-0.26,1.25,38.84,XY24,NTT,ENGINEERING
74,STARHEALTH,551.56,761.0,4.66,M-SC,95.70,171009.0,-29207.0,105239.0,0.86,-14.59,61.54,37.97,195.0,-0.28,1.31,37.31,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.66
1,25,42.09
2,50,70.54


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.68
AR      15.01
XY25    14.60
X40N    13.30
X40     12.48
XR      10.36
X200     1.43
SR       1.13
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.06
X-LC    18.03
M-SC    17.45
H-LC     9.15
H-MC     8.52
M-LC     7.71
X-MC     5.55
M-MC     5.52
X-SC     2.76
L-SC     2.30
L-LC     1.12
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.46,-1.62,33.43
IT,10.20,-16.76,80.18
BANKS,9.64,-10.96,61.97
MISC,6.54,-25.91,81.61
FINANCE,5.83,-25.32,84.83
INSURANCE,5.30,3.83,25.00
PAINTS,5.30,-18.45,53.51
ELECTRICAL,5.11,-4.98,50.52
HEALTHCARE,4.36,-4.32,35.45


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2824919.0,26
AR,1258109.0,14
X40N,1208161.0,17
XR,1094474.0,14
X40,773940.0,9
XY25,464999.0,10
SR,196769.0,2
X200,71969.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2212160.0,20
M-SC,2034302.0,21
X-LC,849543.0,11
M-MC,543531.0,5
H-MC,511536.0,8
X-SC,346745.0,4
X-MC,305873.0,5
L-SC,296934.0,4
M-LC,295729.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      922620.0      6
M-SC       XY24      754427.0      6
H-SC       AR        540794.0      3
           X40N      498934.0      6
X-LC       X40       480496.0      4
M-SC       X40N      471046.0      6
M-MC       XY24      386169.0      3
M-SC       XR        289018.0      4
H-LC       AR        247121.0      5
H-SC       XR        237838.0      4
M-SC       X40       219579.0      2
M-LC       XY24      174643.0      4
H-MC       XY24      170871.0      2
X-LC       AR        167990.0      2
L-SC       XR        157283.0      2
X-MC       XY24      137270.0      2
X-LC       XY25      129088.0      4
M-SC       XY25      121502.0      1
X-SC       X40N      118777.0      2
           XR        115837.0      1
H-MC       XR        113370.0      1
X-SC       SR        112131.0      1
M-LC       XR        106425.0      1
M-SC       AR         94092.0      1
X-MC       X40N       93224.0      2
M-SC       SR         84638.0      1
M-MC       AR         82659.0      1
L-SC       XY24       78228.0      1
X-MC       XY25       75379.0      1
H-MC       XY25       75194.0      1
M-MC       XR         74703.0      1
X-LC       X200       71969.0      1
H-MC       AR         64030.0      1
           X40        61891.0      2
L-SC       AR         61423.0      1
L-LC       XY25       42974.0      1
H-MC       X40N       26180.0      1
M-LC       XY25       14661.0      1
H-SC       X40        11974.0      1
H-LC       XY25        6201.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
